<div align="right" dir="auto">
   
#  دیتاست اول(COVID)
 </div>
  


<div align="right" dir="auto">
   
##  اتصال به کلاستر اسپارک و هدوپ 

    در این قسمت از تمرین باید به عنوان درایور یک سسشن  به کلاستر اسپارک بسازیم.
 </div>
  
* **appName**: application name displayed at the Spark Master Web UI;
* **master**: Spark Master URL, same used by Spark Workers;
* **spark.executor.memory**: must be less than or equals to docker compose SPARK_WORKER_MEMORY config.

In [1]:
pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 1.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 4.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.8/285.8 kB 4.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 4.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 4.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 4.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 5.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyspark
from pyspark.sql import SparkSession

import os
import pandas
from pyspark.sql.types import *

In [3]:
spark = SparkSession.builder.appName('spark-notebook-1').master('spark://spark-master:7077').config('spark.executor.memmory', '2G').getOrCreate()
spark

2024-01-10 17:19:46,040 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).



<div align="right" dir="auto">
   
#  خواندن داده 

    :در اسپارک ما ساختارهای مختلفی برای کار با داده و پخش شدن آن‌ها در شبکه داریم که به ۳ دسته تقسیم بندی میشوند 
+ RDD
+ Dataset
+ DataFrame
    
    
     برای مطالعه بیشتر به لینک زیر مراجعه کنید:
[rdd-vs-dataframe-vs-dataset](https://phoenixnap.com/kb/rdd-vs-dataframe-vs-dataset)

    ما در درس با ساختار RDD آشنا شدیم حال در این تمرین میخواهیم با ساختار Dataframe آشنا شویم و به کمک آن دیتا را از روی HDFS بخوانیم و روی آن فایل‌ها پردازش انجام دهیم
 
 </div>
  
    

In [4]:
# The data from covid.csv will be loaded from a local file path
for path, subdirs, files in os.walk('./data/'):
    for name in files:
        if "covid" in name:
            csvName = name
            csvPath = os.path.join(path, name)
            print("Loading data from csv {}".format(csvPath))
            covidDfPandas = pandas.read_csv(csvPath)

Loading data from csv ./data/covid.csv
Loading data from csv ./data/.ipynb_checkpoints/covid-checkpoint.csv


In [5]:
# Structure of data in the DataFrame
schema = StructType([
    StructField("dateRep", StringType(), True),
    StructField("day", IntegerType(), True),
    StructField("month", IntegerType(), True),
    StructField("year", IntegerType(), True),
    StructField("cases", IntegerType(), True),
    StructField("deaths", IntegerType(), True),
    StructField("countriesAndTerritories", StringType(), True),
    StructField("geoId", StringType(), True),
    StructField("countryterritoryCode", StringType(), True),
    StructField("popData2019", FloatType(), True),
    StructField("continentExp", StringType(), True),
    StructField("Cumulative_number_for_14_days_of_COVID-19_cases_per_100000", DoubleType(), True),
])


<div align="right" dir="auto">
یک DataFrame از Pandas را به یک DataFrame از Spark تبدیل کنید و سپس طرح ساختاری و برخی از داده‌ها را نمایش دهید.
 </div>
  
    




In [6]:
# Convert Pandas DataFrame to Spark DataFrame with defined schema
covidDfSpark = spark.createDataFrame(covidDfPandas.values.tolist(), schema=schema)

# Show the DataFrame schema and some data
covidDfSpark.printSchema()

root
 |-- dateRep: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- cases: integer (nullable = true)
 |-- deaths: integer (nullable = true)
 |-- countriesAndTerritories: string (nullable = true)
 |-- geoId: string (nullable = true)
 |-- countryterritoryCode: string (nullable = true)
 |-- popData2019: float (nullable = true)
 |-- continentExp: string (nullable = true)
 |-- Cumulative_number_for_14_days_of_COVID-19_cases_per_100000: double (nullable = true)




<div align="right" dir="auto">
   
#  اسپارک SQL 

    مهمترین قابلیت اسپارک این است که می‌تواند با خواندن فایل‌ها به صورت توزیع شده روی آن‌ها پردازش انجام دهد و این پردازش را برنامه ‌نویس میتواند با استفاده از دستورات SQL اعمال کند
    در این بخش از شما انتظار می‌رود که به وسیله spark SQL  به اسپارک کوئری  بزنید . 
    
    
 </div>

[pyspark.sql.functions.col](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.col.html)

[pyspark.sql.DataFrame.count](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.count.html)

[pyspark.sql.DataFrame.filter](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.filter.html)

[pyspark.sql.DataFrame.groupBy](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.groupBy.html)

<div align="right" dir="auto">
برای اینکه بتوانیم روی دیتای لود شده به وسیله تابع  spark.sql
    کوئری‌های SQL بزنیم
    باید دو دیتاست لود شده را به عنوان table
    به spark 
    معرفی کنیم
</div>

In [7]:
covidDfSpark.registerTempTable("covid")

<div align="right" dir="auto">
 ده رکورد آخر را نمایش دهید
</div>

In [8]:
last_10_records = spark.sql("SELECT * FROM covid ORDER BY dateRep DESC LIMIT 10")
last_10_records.show()

2024-01-10 17:19:54,294 WARN scheduler.TaskSetManager: Stage 0 contains a task of very large size (1650 KiB). The maximum recommended task size is 1000 KiB.
ERROR:root:KeyboardInterrupt while sending command.                 (1 + 1) / 2]
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1217, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

<div align="right" dir="auto">
اسکیما یا ساختار دیتاست ها را نمایش دهید

</div>

In [ ]:
# to-do
spark.sql("DESCRIBE EXTENDED covid;").show()

In [ ]:
covidDfSpark.printSchema()

<div align="right" dir="auto">
داده‌های مربوط به ستون   countryterritoryCode ,deaths را پرینت کنید(۱۰تا) 

</div>

In [ ]:
# to-do
spark.sql("SELECT countryterritoryCode, deaths FROM covid LIMIT 10").show()

<div align="right" dir="auto">
تعداد کل موارد Covid برابر است با?
</div>

In [ ]:
# to-do
spark.sql("SELECT SUM(cases) AS total_cases FROM covid").show()

<div align="right" dir="auto">
مجموع موارد کروناویروس در اکتبر ۲۰۲۰ در بلژیک را به دست آورید.
</div>



In [ ]:
# to-do
covid_cases_belgium_october_2020 = spark.sql(
    "SELECT SUM(cases) AS total_cases_october_2020_belgium \
    FROM covid \
    WHERE month = 10 AND year = 2020 AND countriesAndTerritories = 'Belgium'"
).show()


<div align="right" dir="auto">
جمع تعداد موارد و مرگ‌ها در کشورهای آسیا به دست آورید.
 
 </div>


In [ ]:
# to-do
total_cases_deaths_by_country_asia = spark.sql(
    "SELECT countriesAndTerritories, SUM(cases) AS total_cases, SUM(deaths) AS total_deaths \
    FROM covid \
    WHERE continentExp = 'Asia' \
    GROUP BY countriesAndTerritories"
).show()


<div align="right" dir="auto">
میانگین تعداد موارد و مرگ‌ها در کشورهای آسیا به دست آورید.
 
 </div>


In [ ]:
# to-do
spark.sql(
    "SELECT countriesAndTerritories, AVG(cases) AS average_cases, AVG(deaths) AS average_deaths \
    FROM covid \
    WHERE continentExp = 'Asia' \
    GROUP BY countriesAndTerritories"
).show()


<div align="right" dir="auto">
جمع مرگ‌های کووید در اروپا بر اساس تاریخ به دست آورید.
 
 </div>

In [ ]:
# to-do
spark.sql(
    "SELECT dateRep, SUM(deaths) AS total_deaths \
    FROM covid \
    WHERE continentExp = 'Europe' \
    GROUP BY dateRep \
    ORDER BY dateRep"
).show()


<div align="right" dir="auto">
   
#  اسپارک toPandas 

    یکی ار قابلیت‌های اسپارک این است که می‌توان dataframe های آن را به 
    dataframe های pandas تبدیل کند و از توابع آن از جمله
    توابع plot  آن برای رسم نمودار استفاده کرد.
    در این قسمت از شما انتظار می‌رود نمودار تعداد جنگ‌ها بر اساس هر گونه را رسم کنید.
    
 </div>
  
    

  
[pyspark.sql.DataFrame.toPandas](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.toPandas.html)

[pyspark.pandas.DataFrame.plot](https://spark.apache.org/docs/3.2.1/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.plot.html)



<div align="right" dir="auto">
جمع موارد و مرگ‌ها برای هر قاره را به دست آورید.
 
 </div>

In [ ]:
cases_deaths_by_continent = spark.sql(
    "SELECT continentExp, SUM(cases) AS total_cases, SUM(deaths) AS total_deaths \
    FROM covid \
    GROUP BY continentExp"
)

pandas_df = cases_deaths_by_continent.toPandas()

pandas_df.plot(kind='bar', x='continentExp', y=['total_cases', 'total_deaths'], title='Total Cases and Deaths by Continent')

<div align="right" dir="auto">
نمودار مبتلایان در طور زمان را برای کشور ایران را بنویسید. 
    
 </div>
  

In [ ]:
import pandas as pd

covid_iran = covidDfSpark.filter(covidDfSpark.countriesAndTerritories == 'Iran')

covid_iran_pandas = covid_iran.toPandas()

covid_iran_pandas['dateRep'] = pd.to_datetime(covid_iran_pandas['dateRep']) 
covid_iran_pandas.sort_values(by='dateRep', inplace=True) 

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(covid_iran_pandas['dateRep'], covid_iran_pandas['cases'], color='purple')
plt.title('COVID-19 Cases Over Time in Iran')
plt.xlabel('Date')
plt.ylabel('Number of Cases')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Assuming you have a DataFrame named covidDfSpark
covid_iran.select('dateRep').show(20)


<div align="right" dir="auto">
   
#  اسپارک Shuffle 

    Spark SQL shuffle مکانیزمی است برای توزیع مجدد یا پارتیشن بندی مجدد داده ها به طوری که داده ها به طور متفاوت در پارتیشن ها گروه بندی می شوند، بر اساس اندازه داده شما ممکن است نیاز باشد تعداد پارتیشن های RDD/DataFrame را با استفاده از اسپارک کاهش یا افزایش دهید.
    برای مثال وقتی روی دو dataframe مختلف که روی شبکه توزیع شده اند
    دستور join را میزنیم یک عملیات 
    shuffling انجام میشود
    در این قسمت از شما انتظار می‌رود کوئری جوین زیر را نوشته و اجرا کنید همچنین به  صفحه 
    application master ui
    مراجعه کنید و نحوه shuffleing را گزارش کنید . 
     و همچنین توضیح دهید DAG scheduler  در اسپارک چیست ؟
 </div>
  
    

  
[shuffling in standalone cluster](https://medium.com/@rachit1arora/apache-spark-shuffle-service-there-are-more-than-one-options-c1a8e098230e)





<div align="right" dir="auto">
از desc در PySpark برای مرتب‌سازی نزولی استفاده  کرده و با استفاده از alias یک self-join روی DataFrame covidDfSpark انجام دهید، سپس با گروه‌بندی بر اساس ستون a.continentExp از داده‌های پیوسته، تعداد وقوع‌ها شمرده شده و نتایج نمایش دهید. 
</div>



In [ ]:
from pyspark.sql import functions as F

covidDfSpark.alias("a").join(covidDfSpark.alias("b"), "continentExp") \
    .groupBy("a.continentExp") \
    .agg(F.sum("a.cases").alias("total_cases")) \
    .orderBy(F.desc("continentExp")) \
    .show()


In [ ]:
import time

epochNow = int(time.time())

<div align="right" dir="auto">
نوشتن یکDataFrame به عنوان یک فایل parquet در HDFS
 
 </div>

In [ ]:
covidDfSpark.write.parquet("hdfs://namenode:8020/covid/{}_{}.parquet".format(csvName,epochNow))
print("Covid Dataframe stored in Hadoop.")

In [ ]:
# Read from HDFS to confirm it was successfully stored

df_load = spark.read.parquet("hdfs://namenode:8020/covid/{}_{}.parquet".format(csvName,epochNow))
print("Covid Dataframe read from Hadoop : ")
df_load.show()


<div align="right" dir="auto">
   
#  دیتاست دوم(پترن‌های باکتری Ecoli)
 </div>
  

In [ ]:
# Create a new SparkSession
spark2 = SparkSession.builder.appName('spark-notebook-2').master('spark://spark-master:7077').getOrCreate()

# Create the schema
schema = StructType([
    StructField("0", StringType(), True)])

In [ ]:
# The data from covid.csv will be loaded from a local file path
for path, subdirs, files in os.walk('./data/'):
    for name in files:
        if "ecoli" in name and name.endswith(".txt"):
            txtPath = os.path.join(path, name)
            print("Loading data from text file: {}".format(txtPath))
            # Load the ecoli.txt file
            lines_txt = pandas.read_csv(txtPath)

In [ ]:
# Create a new SparkSession
spark2 = SparkSession.builder.appName('spark-notebook-2').master('spark://spark-master:7077').getOrCreate()

# Create the schema
schema = StructType([StructField("0", StringType(), True)])

In [ ]:
lines_sp = spark2.createDataFrame(lines_txt, schema=schema)
# Preview the structure
lines_sp.show(10)

<div align="right" dir="auto">
   
#  اسپارک RDD 

    
 </div>
  

<div align="right" dir="auto">
DataFrame را به RDD تبدیل کنید.
 </div>

In [ ]:
# to-do
lines_rdd = lines_sp.rdd

In [ ]:
# Define the lengths for patterns (3 and 4)
pattern_lengths = [3, 4]
lines_rdd = lines_rdd.map(lambda row: row[0])
lines_rdd

In [ ]:
def line_to_grams(line, length):
    ngrams = []
    for i in range(len(line) - length + 1):
        ngrams.append(line[i : i + length])
    return ngrams

pattern_lengths = [3, 4]

<div align="right" dir="auto">
ایجاد RDD برای هر طول الگو (n-gram):
 
ابتدا یک لیست از RDDهای n-gramها برای هر طول الگو (pattern length) ایجاد کرده.
برای هر طول الگو، با استفاده از flatMap بر روی هر خط موجود در مجموعه داده (lines_rdd)، n-gramهای مربوط به طول الگو مورد نظر را ایجاد کرده و به لیستی از RDDها اضافه کنید.
 </div>
  

In [ ]:
ngrams_rdds = []
for length in pattern_lengths:
    #to-do
    ngram_rdd = lines_rdd.flatMap(lambda line: line_to_grams(line, length))
    ngrams_rdds.append(ngram_rdd)    
ngrams_rdds


<div align="right" dir="auto">
 محاسبه تعداد n-gramها:
سپس، برای هر RDD حاصل از مرحله قبل، تعداد تکرارهای هر n-gram را محاسبه کرده.
این کار با استفاده از map و reduceByKey بر روی هر n-gram، جفت‌هایی از (n-gram, تعداد تکرار) را ایجاد کرده و سپس نتایج را به یک لیست از RDDها اضافه کرده.
در نهایت، دو RDD ایجاد شده داده‌های مربوط به هر طول الگو را در اختیار دارند.
     </div>

In [ ]:
count_rdds = []

def count_ngrams(rdd):
    return rdd.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

# Calculate counts of n-grams for each RDD and pattern length
for ngrams_rdd, length in zip(ngrams_rdds, pattern_lengths):
    # Calculate counts for each n-gram in the RDD using map and reduceByKey
    ngram_counts = count_ngrams(ngrams_rdd)
    # Append the RDD containing n-gram counts to the list
    count_rdds.append(ngram_counts)
count_rdds 

In [ ]:
merged_rdd = count_rdds[0].union(count_rdds[1])
dna_patterns = merged_rdd.sortByKey(ascending=False).map(lambda x: f"{x[1]} {x[0]}")

In [ ]:
dna_patterns.take(10)

In [ ]:
# Split each string into two columns (ngram and count) using space as the delimiter
split_rdd = dna_patterns.map(lambda line: line.split())

# Create a DataFrame from the RDD with column names
columns = ["pattern", "count"]
df = spark2.createDataFrame(split_rdd, columns)
split_rdd

In [ ]:
df.show(10)


<div align="right" dir="auto">
   
# تابع تعریف شده توسط کاربر(UDF)

یکی از مزایای اسپارک این است که نه تنها برای ما یک زبان SQL فراهم کرده که روی چندین سرور به صورت همزمان پردازش را انجام دهد بلکه 
    میتوان به زبان‌های مختلف توابعی تعریف کرد که روی  همه executor ها اجرا شود   در این بخش از شما انتظار می‌رود که با نوشتن یک UDF به زبان پایتونی    توضیحات  قسمت  قبل را انجام دهید.

 </div>
 
[pyspark.sql.functions.udf](https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html)

[pyspark.sql.DataFrame.withColumn](https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.DataFrame.withColumn.html)

    



In [ ]:
def split_string(line):
    parts = line.split()  # Splitting assuming space as delimiter
    return parts[0], int(parts[1]) if len(parts) > 1 else None

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.functions import udf

# Define the UDF to split the string into two columns
# to-do
returnType = StructType([StructField("ngram", StringType(), True),StructField("count", IntegerType(), True)])
split_string_udf = udf(split_string, returnType)

# Apply the UDF to create separate columns
df_with_split = df.withColumn("split", split_string_udf(df["count"])).select("split.ngram")

# Show the resulting DataFrame
# to-do
df_with_split.show()